In [ ]:
# test = pd.read_csv('../data/notebook_produced_3/test.csv')

# success_test = test.loc[test.poutcome_success==1, :]
# success_test = success_test.drop(columns=['poutcome_failure', 'poutcome_success', 'pdays'])

# X_test = success_test.drop(columns='y')
# y_test = success_test.y

# X_test.shape, y_test.shape


# roc_auc_score(y_test, model.predict(X_test))

# print(classification_report(y_test, model.predict(X_test)))

# plot_confusion_matrix(model, X_test, y_test,
#                       display_labels=['no', 'yes'],
#                       cmap=plt.cm.Blues,
#                       normalize='true'
#                      );



# clf = joblib.load('../saved_models/random_forest_feature_selected.joblib')

In [ ]:
# def tp_tn_fn_fp(true_value, predicted_value):
#     tp, tn, fn, fp = 0, 0, 0, 0
#     for v, p in zip(true_value, predicted_value):
#         if v == p:
#             if v+p == 2: tp += 1
#             else: tn += 1
#         elif v == 1: fn += 1
#         else: fp += 1
#     return tp, tn, fn, fp

# def lift_score(true_value, predicted_value):
#     tp, tn, fn, fp = tp_tn_fn_fp(true_value, predicted_value)
#     return ( tp/(tp+fp) ) / ( (tp+fn) / (tp+tn+fp+fn) )

# lift_scorer = make_scorer(lift_score)

# lift_score(y_holdout, clf.predict(X_holdout))



# y_probs = clf.predict_proba(X_holdout)[:,1]
# y_pred = clf.predict(X_holdout)

# res = pd.DataFrame(np.column_stack((y_holdout, y_probs, y_pred)),
#                    columns=['target', 'probability', 'prediction'])
# res['decile_group'] = pd.qcut(res.probability, 10, labels=['dec 1', 'dec 2', 'dec 3', 'dec 4', 'dec 5', 'dec 6', 'dec 7', 'dec 8', 'dec 9', 'dec 10'])


# import seaborn as sns

# g = res.groupby(['decile_group', 'target']).count()
# sns.barplot(data=g.reset_index(), x='decile_group', y='probability', hue='target');

# population = res.target.value_counts()
# population_ratio = population.loc[1.0]/population.sum()

# l = []
# for dec_group in res.decile_group.unique():
#     dec = res.loc[res.decile_group==dec_group, :]
#     sample = dec.target.value_counts()
#     sample_ratio = sample.loc[1.0]/sample.sum()
#     l.append([dec_group, sample_ratio/population_ratio])
# deciles = pd.DataFrame(l, columns=['decile', 'lift'])

# l = []
# for i in np.arange(0, 1, .1):
#     d = res.loc[res.probability >= i,:]
#     sample = d.target.value_counts()
#     sample_ratio = sample.loc[1.0]/sample.sum()
#     l.append([round(i, ndigits=1), sample_ratio/population_ratio])
# above_prob = pd.DataFrame(l, columns=['above_probability', 'lift'])

# f, (ax1, ax2) = plt.subplots(ncols=2, figsize=(20,4))
# sns.barplot(data=above_prob, x='above_probability', y='lift', ax=ax1)
# sns.barplot(data=deciles, x='decile', y='lift', ax=ax2)
# ax1.hlines(1, -1, 10)
# ax2.hlines(1, -1, 10);

# test.loc[:, 'job_retired':].head()